# DB2-Salesforce populate survey URLs obtain from DB2

In [1]:
# Parameters
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# login parameters to be handled by Papermill
from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')

salesforce = application.new_salesforce_engine()
db_s = salesforce

In [2]:
import pandas as pd
import datetime

## Get the URL code from DB2


In [3]:
# page_id indicates the question
url_code_df = pd.read_sql_query("select user_id, code, expiration from jos_reply_access_codes \
where page_id = 2 order by user_id desc;",nanohub_db) #limit 5000

new_pref_code_df = pd.read_sql_query("select user_id, code, expiration from jos_reply_access_codes \
where page_id = 1 order by user_id desc;",nanohub_db) # limit 5000

In [4]:
display(url_code_df.head(2))

,user_id,code,expiration
0,326091,96f633224474ac9d16f417df63655c90bca5b46d1467a0...,2021-09-12
1,326090,6dfb91f7a3a71dc7fe7f591d2b58f32dc232042baa8c14...,2021-09-12


In [5]:
## populating full url codes
url_base = "https://nanohub.org/reply/pages/2?code="

from copy import deepcopy
codes = deepcopy(url_code_df['code'])

#import time
#st = time.time()
codes = codes.apply(lambda x: url_base+x)
#tt = st-time.time()
#print(tt)
print(codes[0])


https://nanohub.org/reply/pages/2?code=96f633224474ac9d16f417df63655c90bca5b46d1467a08ae88a4b6e9d834a7b


In [6]:
## add the new code to the dataframe
url_code_df['code'] = codes
url_code_df = url_code_df.rename(columns={"user_id": "nanoHUB_user_ID__c", "code":"nh_use_survey_link__c"})
display(url_code_df.head(2))

,nanoHUB_user_ID__c,nh_use_survey_link__c,expiration
0,326091,https://nanohub.org/reply/pages/2?code=96f6332...,2021-09-12
1,326090,https://nanohub.org/reply/pages/2?code=6dfb91f...,2021-09-12


In [7]:
## determining the new url survey links
url_base1 = 'https://nanohub.org/reply/email-subscriptions?code='

codes = deepcopy(new_pref_code_df['code'])

codes = codes.apply(lambda x: url_base1+x)

print(codes[0])
new_pref_code_df['code'] = codes

https://nanohub.org/reply/email-subscriptions?code=7bf90e6702a422640c6c56a3fbe135cbb51498061d3195e5ba1b4ab5531dd028


In [8]:
display(new_pref_code_df.head(5))

,user_id,code,expiration
0,326091,https://nanohub.org/reply/email-subscriptions?...,2021-09-12
1,326090,https://nanohub.org/reply/email-subscriptions?...,2021-09-12
2,326089,https://nanohub.org/reply/email-subscriptions?...,2021-09-12
3,326088,https://nanohub.org/reply/email-subscriptions?...,2021-09-12
4,326087,https://nanohub.org/reply/email-subscriptions?...,2021-09-12


In [9]:
# expiration dates can be different, so do a left join instead of "url_code_df['new_preferences_survey_url__c'] = codes"
unif_df = pd.merge(url_code_df,new_pref_code_df,how='left',right_on='user_id',left_on='nanoHUB_user_ID__c')

display(unif_df.head(2))
display(unif_df.tail(2))

,nanoHUB_user_ID__c,nh_use_survey_link__c,expiration_x,user_id,code,expiration_y
0,326091,https://nanohub.org/reply/pages/2?code=96f6332...,2021-09-12,326091,https://nanohub.org/reply/email-subscriptions?...,2021-09-12
1,326090,https://nanohub.org/reply/pages/2?code=6dfb91f...,2021-09-12,326090,https://nanohub.org/reply/email-subscriptions?...,2021-09-12


,nanoHUB_user_ID__c,nh_use_survey_link__c,expiration_x,user_id,code,expiration_y
240617,1000,https://nanohub.org/reply/pages/2?code=ejjJycB...,2021-09-12 02:59:59,1000,https://nanohub.org/reply/email-subscriptions?...,2021-08-18 14:31:42
240618,998,https://nanohub.org/reply/pages/2?code=rWB03ln...,2021-09-12 02:59:59,998,https://nanohub.org/reply/email-subscriptions?...,2021-08-18 14:31:42


In [10]:
# some formatting for SF
unif_df = unif_df.rename(columns={'expiration_x':'nh_use_expire__c',\
                'code':'new_preferences_survey_url__c','expiration_y':'new_prefs_expire__c'})
unif_df = unif_df.drop(columns=['user_id']) #unified_dataframe

In [11]:
display(unif_df.head(2))
display(unif_df.tail(2))

,nanoHUB_user_ID__c,nh_use_survey_link__c,nh_use_expire__c,new_preferences_survey_url__c,new_prefs_expire__c
0,326091,https://nanohub.org/reply/pages/2?code=96f6332...,2021-09-12,https://nanohub.org/reply/email-subscriptions?...,2021-09-12
1,326090,https://nanohub.org/reply/pages/2?code=6dfb91f...,2021-09-12,https://nanohub.org/reply/email-subscriptions?...,2021-09-12


,nanoHUB_user_ID__c,nh_use_survey_link__c,nh_use_expire__c,new_preferences_survey_url__c,new_prefs_expire__c
240617,1000,https://nanohub.org/reply/pages/2?code=ejjJycB...,2021-09-12 02:59:59,https://nanohub.org/reply/email-subscriptions?...,2021-08-18 14:31:42
240618,998,https://nanohub.org/reply/pages/2?code=rWB03ln...,2021-09-12 02:59:59,https://nanohub.org/reply/email-subscriptions?...,2021-08-18 14:31:42


## Send to SF

Obtained Salesforce access token ...... True


In [25]:
db_s.object_id = object_id
db_s.external_id = external_id

# send data to Salesforce
db_s.send_data(unif_df)

[Success] Bulk job creation successful. Job ID = 7505w00000Vrxf9AAB
hello
[Success] CSV upload successful. Job ID = 7505w00000Vrxf9AAB
[Success] Closing job successful. Job ID = 7505w00000Vrxf9AAB


In [39]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000Vrxf9AAB',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-05-07T02:56:04.000+0000',
 'systemModstamp': '2021-05-07T03:06:14.000+0000',
 'state': 'JobComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 240619,
 'numberRecordsFailed': 1129,
 'retries': 0,
 'totalProcessingTime': 2196921,
 'apiActiveProcessingTime': 1921625,
 'apexProcessingTime': 17534312}

In [40]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

('"sf__Id","sf__Error",nanoHUB_user_ID__c,new_preferences_survey_url__c,new_prefs_expire__c,nh_use_expire__c,nh_use_survey_link__c\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","81398.0","https://nanohub.org/reply/email-subscriptions?code=Net3NxOB1D6ELn7VqmDp0PRCIkZGfgM44q234hEmUzabe4vg7JYbmPDnMKx8mmk8","2021-08-18","2021-09-12","https://nanohub.org/reply/pages/2?code=sbiSnv5GENaVvdKsBxgSE1GwNwKDRRUPOLqjUcTSLMlwNTgOjWzGbkKtz6T7YMEk"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","1819.0","https://nanohub.org/reply/email-subscriptions?code=W4Vwc1nSGxfxou5vZDzewJmoIXLNcnS8FtUamVp3gd7PXMhCvbG1fnddQBFGckKL","2021-08-18","2021-09-12","https://nanohub.org/reply/pages/2?code=gadZ6YRCzsTnrBxChNlIn2taLjL5l2RbHl2Vtn6MJXL3r3V6WZbniuVjI5SPCKOM"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","1000.0","https://nanohub.org/reply/email-subscriptions?code=D7SEgOzUiKdwB8YjvraJd2

 '--","305091.0","https://nanohub.org/reply/email-subscriptions?code=17738fe3c078a9d1acc6c0a8de8094df8734ed92dd94f54f4c8881cf0e180d7c","2021-09-12","2021-09-12","https://nanohub.org/reply/pages/2?code=c292c539ae7dd08e73fc93b3b40b1f92030f7147cf2bc90a076be9f08762c11c"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","305085.0","https://nanohub.org/reply/email-subscriptions?code=321537e8a63a8a3a53f19885ec7b42892c1443a725a6958c1fc0100663456418","2021-09-12","2021-09-12","https://nanohub.org/reply/pages/2?code=4f682a20ba003463fca297f960d2fed2dea825b389ceb4f386cba3ae33bd31ce"\n'
 '"","REQUIRED_FIELD_MISSING:Required fields are missing: [LastName]:LastName '
 '--","305061.0","https://nanohub.org/reply/email-subscriptions?code=9b4063ded1ca602ea2ad574c2b93d2e4b745162ca0b707b3588bfe36c087c396","2021-09-12","2021-09-12","https://nanohub.org/reply/pages/2?code=26c7eafb6217ab1ada05bb404c46809a9c68a4e55713cb38e525180770360d5c"\n'
 '"","REQUIRED_FIELD_MISSING:R